In [1]:
from __future__ import annotations
import numpy as np, pandas as pd, warnings, os, openai, json
from openai import OpenAI


from typing import TypedDict, Literal, Dict, Any, Optional
import re

from langgraph.graph import StateGraph, END

In [2]:
# =========================
# 1) State 정의
# =========================
Specialty = Literal["산부인과", "소아청소년과", "응급의학과", "내과", "일반"]

class GraphState(TypedDict, total=False):
    question: str
    specialty: Specialty
    answer: str
    debug: Dict[str, Any]


# =========================
# 2) Specialty별 시스템 프롬프트(예시)
# =========================
SPECIALTY_SYSTEM_PROMPTS: Dict[Specialty, str] = {
    "산부인과": (
        "당신은 산부인과 전문 지식에 기반해 환자 질문에 답하는 의료 상담 도우미입니다.\n"
        "- 임신 가능성, 생리/부정출혈, 골반통, 질 분비물/가려움, 성병 가능성 등 고려\n"
        "- 위험 신호(심한 출혈, 실신, 극심한 통증, 임신 중 출혈 등) 시 응급 권고\n"
        "- 진단이 아닌 정보 제공으로, 필요한 경우 병원 방문/검사 권고\n"
        "출력 형식:\n"
        "1) 가능한 원인(우선순위)\n2) 추가로 확인할 질문\n3) 집에서 할 수 있는 조치\n4) 즉시 진료/응급실이 필요한 경우\n"
    ),
    "소아청소년과": (
        "당신은 소아청소년과 상담 도우미입니다.\n"
        "- 연령/체중/접종력/기저질환 고려\n"
        "- 탈수, 호흡곤란, 고열 지속, 의식 저하 등 위험 신호 강조\n"
        "출력 형식:\n"
        "1) 가능성 높은 원인\n2) 집에서 관찰 포인트\n3) 병원 방문 기준(응급 포함)\n"
    ),
    "응급의학과": (
        "당신은 응급의학과 트리아지(중증도 판단) 중심 상담 도우미입니다.\n"
        "- 생명 위협/즉시 처치가 필요한 위험 신호를 먼저 확인\n"
        "- 빨리 병원/응급실이 필요한 경우를 명확히 안내\n"
        "출력 형식:\n"
        "1) 지금 당장 확인할 위험 신호 체크리스트\n2) 응급실/119 기준\n3) 응급이 아닐 때 다음 단계\n"
    ),
    "내과": (
        "당신은 내과 상담 도우미입니다.\n"
        "- 복통, 소화기, 감염, 만성질환(당뇨/고혈압 등) 관점에서 설명\n"
        "- 감별진단은 단정하지 말고 가능성/다음 검사/경고증상 중심\n"
        "출력 형식:\n"
        "1) 가능한 원인(우선순위)\n2) 추가 질문(증상/기간/동반증상)\n"
        "3) 자가 관리\n4) 진료 필요 기준(응급 포함)\n"
    ),
    "일반": (
        "당신은 일반 의료 정보 제공 도우미입니다.\n"
        "- 전문과 판별이 어려운 경우에도 안전하게 안내\n"
        "- 경고 증상과 다음 행동(어떤 과로 갈지)을 제시\n"
        "출력 형식:\n"
        "1) 가능성/설명\n2) 추가 질문\n3) 다음 행동(어떤 과/검사)\n4) 위험 신호\n"
    ),
}



In [6]:
# =========================
# 3) LLM 호출부 (플러그형)
# =========================
# 아래는 "어떤 LLM을 쓰든" 교체 가능하도록 어댑터 형태로 작성.
# 예: langchain_openai.ChatOpenAI, Azure, Ollama, etc.

class LLMClient:
    """아주 간단한 LLM 인터페이스(교체용)."""
    def __init__(self, model):
        self.model = model

    def invoke(self, system_prompt: str, user_text: str) -> str:
        # LangChain ChatModel 스타일을 가정한 예시.
        # 사용하는 모델에 맞게 수정하면 됨.
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_text},
        ]
        resp = self.model.invoke(messages)
        # resp가 문자열이거나 AIMessage일 수 있으니 안전 처리
        return getattr(resp, "content", resp)



In [21]:
# =========================
# 4) 분류기 노드: (A) 키워드 기반(즉시 동작)
# =========================
def classify_specialty_keyword(state: GraphState) -> GraphState:
    q = state["question"].strip()

    # 매우 단순한 예시 규칙(필요하면 확장)
    obgyn_kw = ["임신", "생리", "월경", "부정출혈", "질", "냉", "골반", "자궁", "난소", "피임"]
    peds_kw = ["아기", "신생아", "유아", "어린이", "초등", "청소년", "아이", "소아", "예방접종"]
    er_kw = ["호흡곤란", "흉통", "의식", "실신", "경련", "피토", "토혈", "혈변", "심한 출혈", "마비", "극심한 통증"]
    im_kw = ["복통", "설사", "변비", "속쓰림", "구토", "발열", "기침", "가래", "피로", "어지럼", "소화", "위", "장"]

    def has_any(keywords):
        return any(k in q for k in keywords)

    if has_any(er_kw):
        specialty: Specialty = "응급의학과"
    elif has_any(obgyn_kw):
        specialty = "산부인과"
    elif has_any(peds_kw):
        specialty = "소아청소년과"
    elif has_any(im_kw):
        specialty = "내과"
    else:
        specialty = "일반"

    return {
        **state,
        "specialty": specialty,
        "debug": {**state.get("debug", {}), "classifier": "keyword", "matched_specialty": specialty},
    }



In [22]:
# =========================
# 5) 분류기 노드: (B) LLM 기반(권장: 정확도↑)
# =========================
def make_classify_specialty_llm(llm: LLMClient):
    system = (
        "너는 의료 질문을 진료과로 라우팅하는 분류기다.\n"
        "가능한 라벨은 정확히 다음 5개만 사용한다:\n"
        "- 산부인과\n- 소아청소년과\n- 응급의학과\n- 내과\n- 일반\n\n"
        "규칙:\n"
        "1) 응급 신호(의식저하, 호흡곤란, 흉통, 심한 출혈 등)가 있으면 응급의학과 우선\n"
        "2) 임신/생리/질/골반 통증 등은 산부인과\n"
        "3) 소아/청소년(아이, 아기 등)은 소아청소년과\n"
        "4) 복통/소화기/감염/만성질환 중심은 내과\n"
        "5) 애매하면 일반\n\n"
        "출력은 라벨 1개만, 다른 문장 금지."
    )

    def _node(state: GraphState) -> GraphState:
        q = state["question"].strip()
        raw = llm.invoke(system, q).strip()

        # 안전하게 라벨만 정규화
        label_map = {
            "산부인과": "산부인과",
            "소아청소년과": "소아청소년과",
            "응급의학과": "응급의학과",
            "내과": "내과",
            "일반": "일반",
        }
        specialty: Specialty = label_map.get(raw, "일반")

        return {
            **state,
            "specialty": specialty,
            "debug": {**state.get("debug", {}), "classifier": "llm", "raw": raw, "normalized": specialty},
        }

    return _node



In [23]:


# =========================
# 6) Specialty 응답 노드 (프롬프트 분기)
# =========================
def make_specialty_answer_node(specialty: Specialty, llm: LLMClient):
    def _node(state: GraphState) -> GraphState:
        q = state["question"].strip()
        sys_prompt = SPECIALTY_SYSTEM_PROMPTS[specialty]
        
        answer = llm.invoke(sys_prompt, q)

        return {
            **state,
            "answer": answer,
        }
    return _node


# =========================
# 7) Router 함수: conditional edge에서 사용
# =========================
def route_by_specialty(state: GraphState) -> Specialty:
    # 분류 결과가 없거나 이상하면 일반으로
    sp = state.get("specialty", "일반")
    if sp not in ("산부인과", "소아청소년과", "응급의학과", "내과", "일반"):
        return "일반"
    return sp  # type: ignore


# =========================
# 8) 그래프 빌드 함수
# =========================
def build_graph(llm_model, use_llm_classifier: bool = False):
    """
    llm_model: LangChain ChatModel(예: ChatOpenAI, OllamaChat 등) 객체
    use_llm_classifier: True면 LLM 기반 분류기 사용, False면 키워드 분류기 사용
    """
    llm = LLMClient(llm_model)

    g = StateGraph(GraphState)

    # 분류 노드 선택
    if use_llm_classifier:
        g.add_node("classify_specialty", make_classify_specialty_llm(llm))
    else:
        g.add_node("classify_specialty", classify_specialty_keyword)

    # 각 분기 노드
    g.add_node("산부인과", make_specialty_answer_node("산부인과", llm))
    g.add_node("소아청소년과", make_specialty_answer_node("소아청소년과", llm))
    g.add_node("응급의학과", make_specialty_answer_node("응급의학과", llm))
    g.add_node("내과", make_specialty_answer_node("내과", llm))
    g.add_node("일반", make_specialty_answer_node("일반", llm))

    # Entry
    g.set_entry_point("classify_specialty")

    # Conditional routing
    g.add_conditional_edges(
        "classify_specialty",
        route_by_specialty,
        {
            "산부인과": "산부인과",
            "소아청소년과": "소아청소년과",
            "응급의학과": "응급의학과",
            "내과": "내과",
            "일반": "일반",
        },
    )

    # 각 노드는 END로
    g.add_edge("산부인과", END)
    g.add_edge("소아청소년과", END)
    g.add_edge("응급의학과", END)
    g.add_edge("내과", END)
    g.add_edge("일반", END)

    return g.compile()


# =========================
# 9) 사용 예시
# =========================
# if __name__ == "__main__":
#     # 예시: ChatOpenAI를 쓴다면 (사용 환경에 맞게 교체)
#     # from langchain_openai import ChatOpenAI
#     # model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

#     # 여기서는 model 변수만 "네가 쓰는 LLM"로 넣어주면 됨.
#     model = None  # TODO: 네 LLM 객체로 교체

#     app = build_graph(model, use_llm_classifier=False)

#     input_state: GraphState = {"question": "내가 복통이 있는데 왼쪽 아랫배가 아파."}
#     result = app.invoke(input_state)

#     print("specialty =", result.get("specialty"))
#     print("answer =", result.get("answer"))
#     print("debug =", result.get("debug"))


In [24]:

import numpy as np, pandas as pd, warnings, os, openai, json
from tqdm.auto import tqdm
from openai import OpenAI
warnings.filterwarnings('ignore')
# import requests
from typing import List, Tuple, Union
from langchain_ollama.embeddings import OllamaEmbeddings
# import pickle
from sklearn.metrics.pairwise import cosine_similarity
# from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnableLambda
from langchain.memory import ConversationBufferMemory
# from langchain_ollama.chat_models import 
# from langchain.schema import Sys
from langgraph.graph import StateGraph, END
from chromadb import Documents, EmbeddingFunction, Embeddings
import chromadb
from typing import TypedDict, Literal, Dict, Any, Optional

key = open('../../../../api_key.txt','r')
api_key = key.read()
openai.api_key = api_key

base_ = open('../../../../base_url.txt','r')
base_url = base_.read()
# openai.api_key = api_key
def invoke(
        #    prompt,
           system_prompt,
           user_text,
           api_key=api_key,
           model="gpt-4.1",
           temperature=0):
    client = OpenAI(api_key=api_key)

    # 메시지 구성
    messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_text},
        ]
    # 스트리밍 요청
    stream = client.chat.completions.create(
        model=model,
        messages=messages,
        # messages=prompt,
        temperature=temperature,
        # stream=True  # ⭐ 핵심 옵션!
    )

    # generator 반환 (chunk 단위 텍스트 출력)
    # return stream
    return getattr(stream, "content", stream)

memory = ConversationBufferMemory(return_messages=True)

In [25]:
app = build_graph(invoke)

In [26]:
a = app.get_graph()

In [27]:
a


Graph(nodes={'__start__': Node(id='__start__', name='__start__', data=RunnableCallable(tags=None, recurse=True, explode_args=False, func_accepts_config=False, func_accepts={}), metadata=None), 'classify_specialty': Node(id='classify_specialty', name='classify_specialty', data=classify_specialty(tags=None, recurse=True, explode_args=False, func_accepts_config=False, func_accepts={}), metadata=None), '산부인과': Node(id='산부인과', name='산부인과', data=산부인과(tags=None, recurse=True, explode_args=False, func_accepts_config=False, func_accepts={}), metadata=None), '소아청소년과': Node(id='소아청소년과', name='소아청소년과', data=소아청소년과(tags=None, recurse=True, explode_args=False, func_accepts_config=False, func_accepts={}), metadata=None), '응급의학과': Node(id='응급의학과', name='응급의학과', data=응급의학과(tags=None, recurse=True, explode_args=False, func_accepts_config=False, func_accepts={}), metadata=None), '내과': Node(id='내과', name='내과', data=내과(tags=None, recurse=True, explode_args=False, func_accepts_config=False, func_accepts={})

In [28]:

mermaid = app.get_graph().draw_mermaid()
print(mermaid)

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	classify_specialty(classify_specialty)
	____(산부인과)
	______(소아청소년과)
	_____(응급의학과)
	__(내과)
	__(일반)
	__end__([<p>__end__</p>]):::last
	__start__ --> classify_specialty;
	classify_specialty -.-> __;
	classify_specialty -.-> ____;
	classify_specialty -.-> ______;
	classify_specialty -.-> _____;
	classify_specialty -.-> __;
	__ --> __end__;
	____ --> __end__;
	______ --> __end__;
	_____ --> __end__;
	__ --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



In [29]:
png = app.get_graph().draw_mermaid_png()
with open("graph.png", "wb") as f:
    f.write(png)